In [7]:
import numpy as np
import pandas as pd
import datetime
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

In [6]:
# Clean / Load
sentiments = pd.read_csv("../data/sentiment/sentiment_days.csv")
sentiments.date = pd.to_datetime(sentiments.date, format='%Y-%m-%d')
stocks = pd.read_csv("../data/percent_change/relative_change.csv")
stocks.date = pd.to_datetime(stocks.date, format='%Y%m%d')
stocks.drop(['Unnamed: 0'], axis=1, inplace=True)
stocks = stocks[(stocks.date > '2005-12-31') & (stocks.date < '2017-12-31')]

# Join & Label
sentiments['prior_date']=sentiments.date + datetime.timedelta(days=1)
data = stocks.merge(sentiments, left_on='date', right_on='prior_date', how='inner')
data['label'] = np.where(data['rel_difference_close_open']>0, 1, 0)

# Preview
data.head(3)

ticker     date_x  close  rel_difference_close_open  \
0    AAC 2006-01-03  1.640                   0.015244   
1    AAE 2006-01-03  0.365                   0.054795   
2    AAM 2006-01-03  0.255                  -0.098039   

   rel_difference_close_close     date_y     mode  Sum_QDAP       Sum  \
0                    0.051829 2006-01-02  neutral -4.304762  negative   
1                    0.068493 2006-01-02  neutral -4.304762  negative   
2                   -0.078431 2006-01-02  neutral -4.304762  negative   

   Mean_QDAP      Mean prior_date  label  
0  -0.031887  negative 2006-01-03      1  
1  -0.031887  negative 2006-01-03      1  
2  -0.031887  negative 2006-01-03      0

In [9]:
# Get ticker specific data
ticker = 'QAN'
groups = data.groupby('ticker')
tick_dat = groups.get_group(ticker)

In [38]:
# Get the training test data
train, test = train_test_split(tick_dat, test_size=0.2)

train_X = train[['Sum_QDAP']]
train_Y = train[['label']]
test_X = test[['Sum_QDAP']]
test_Y = test[['label']]

# Build the model
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(40, 2), random_state=1)
model = clf.fit(train_X, train_Y.values.ravel())
print(model.score(test_X, test_Y.values.ravel()))

0.5778145695364238
